In [ ]:
import os
import pandas as pd

team_codes = [
    'crd','atl','rav','buf','car','chi','cin','cle','dal','den','det',
    'gnb','htx','clt','jax','kan','sdg','ram','rai','mia','min','nwe',
    'nor','nyg','nyj','phi','pit','sea','sfo','tam','oti','was'
]

data_dir = '/Users/sethfried/Fantasy Football/Cleaned Data/Cleaned_team_Logs'

team_mapping = {
    'crd':'ARI', 'atl':'ATL', 'rav':'BAL', 'buf':'BUF', 'car':'CAR',
    'chi':'CHI', 'cin':'CIN', 'cle':'CLE', 'dal':'DAL', 'den':'DEN',
    'det':'DET', 'gnb':'GNB', 'htx':'HOU', 'clt':'IND', 'jax':'JAX',
    'kan':'KAN', 'sdg':'LAC', 'ram':'LAR', 'rai':'LVR', 'mia':'MIA',
    'min':'MIN', 'nwe':'NWE', 'nor':'NOR', 'nyg':'NYG', 'nyj':'NYJ',
    'phi':'PHI', 'pit':'PIT', 'sea':'SEA', 'sfo':'SFO', 'tam':'TAM',
    'oti':'TEN', 'was':'WAS'
}

opp_replacements = {
    'STL': 'LAR',
    'SDG': 'LAC',
    'OAK': 'LVR',
    'WSH': 'WAS',
    'WFT': 'WAS'
}

all_dfs = []
for code in team_codes:
    path = os.path.join(data_dir, f"{code}_2006_2024_gamelog.csv")
    if not os.path.exists(path):
        print(f"⚠️ File not found: {path}")
        continue

    df_team = pd.read_csv(path, dtype=str)

    df_team['Team'] = team_mapping[code]

    df_team['Opp'] = (
        df_team['Opp']
        .str.upper()
        .str.strip()
        .replace(opp_replacements)
    )

    df_team['Year'] = df_team['Year'].astype(str).str.zfill(4)
    df_team['Week'] = df_team['Week'].astype(str).str.zfill(2)
    df_team['Date'] = df_team['Date'].astype(str)

    all_dfs.append(df_team)

df = pd.concat(all_dfs, ignore_index=True)

for col in ['Year','Week','Date','Team','Opp']:
    df[col] = df[col].str.upper().str.strip()

matchups = set(zip(
    df['Year'], df['Week'], df['Date'],
    df['Team'], df['Opp']
))

df['reciprocal_found'] = df.apply(
    lambda r: (r['Year'], r['Week'], r['Date'], r['Opp'], r['Team']) in matchups,
    axis=1
)

missing = df[~df['reciprocal_found']]
if missing.empty:
    print("✅ All games have matching opponent entries.")
else:
    print(f"❌ {len(missing)} rows with no reciprocal match:")
    print(
        missing[['Year','Week','Date','Team','Opp']]
        .drop_duplicates()
        .sort_values(['Year','Week','Date'])
        .to_string(index=False)
    )


✅ All games have matching opponent entries.


In [121]:
print(df.columns)

Index(['Year', 'Week', 'Date', 'Day', 'Home', 'Opp', 'Score Rslt', 'Team Pts',
       'OT', 'Passing Cmp', 'Passing Att', 'Passing Cmp%', 'Passing Yds',
       'Passing TD', 'Passing Y/A', 'Passing AY/A', 'Passing Rate',
       'Passing Sk', 'Rushing Att', 'Rushing Yds', 'Rushing TD', 'Rushing Y/A',
       'Total Offense Ply', 'Total Offense Tot', 'Total Offense Y/P',
       'Kicking FGA', 'Kicking FGM', 'Kicking XPA', 'Kicking XPM',
       'Punting Pnt', 'Punting Yds', 'First Downs Pass', 'First Downs Rsh',
       'First Downs Pen', 'First Downs 1stD', 'Downs 3DConv', 'Downs 3DAtt',
       'Downs 4DConv', 'Downs 4DAtt', 'Penalties Pen', 'Penalties Yds',
       'Turnovers FL', 'Turnovers Int', 'Turnovers TO',
       'Unnamed: 47_level_0 ToP', 'Opp Pts', 'Team', 'reciprocal_found'],
      dtype='object')


In [122]:
import pandas as pd
import numpy as np
from geopy.distance import geodesic

# 0) Load your cleaned team‐week CSV

# 1) Normalize merge keys
for col in ['Year','Week','Team','Opp']:
    df[col] = df[col].astype(str).str.upper().str.strip()

# 2) List exactly the raw stats you want to pull for opponents
stats = [
    'Team Pts',
    'Passing Yds','Passing Att','Passing Sk',
    'Rushing Yds',
    'Total Offense Tot','Total Offense Ply',
    'Turnovers Int'
]

# 3) Build an opponent‐lookup DataFrame
opp = (
    df[['Year','Week','Team'] + stats]         # each row’s own stats
      .rename(columns={'Team':'Opp'})          # so it lines up with df['Opp']
)
# Prefix those columns:
opp = opp.rename(columns={s: f'Opponent {s}' for s in stats})

# 4) Merge opponent stats back onto the original on Year, Week, Opp
df = df.merge(
    opp,
    on=['Year','Week','Opp'],
    how='left',
    validate='many_to_one'
)

# 5) Coerce all relevant columns to numeric (fill blanks with 0)
to_num = stats + [f'Opponent {s}' for s in stats]
for c in to_num:
    df[c] = pd.to_numeric(df[c], errors='coerce').fillna(0)

# 6) Compute defensive “allowed” metrics
df['PassYdsAllowed']    = df['Opponent Passing Yds']
df['RushYdsAllowed']    = df['Opponent Rushing Yds']
df['PtsAllowed']        = df['Opponent Team Pts']
df['YdsPerPlayAllowed'] = (
    df['Opponent Total Offense Tot'] / df['Opponent Total Offense Ply']
).replace([np.inf, -np.inf], 0).fillna(0)
df['SackRateAllowed']   = (
    df['Opponent Passing Sk'] / df['Opponent Passing Att']
).replace([np.inf, -np.inf], 0).fillna(0)
df['IntRateAllowed']    = (
    df['Opponent Turnovers Int'] / df['Opponent Passing Att']
).replace([np.inf, -np.inf], 0).fillna(0)

# 7) Weekly league‐wide rankings
week_grp = df.groupby(['Year','Week'])
off_stats = ['Passing Yds','Rushing Yds','Total Offense Tot','Team Pts']
def_stats = ['PassYdsAllowed','RushYdsAllowed','PtsAllowed','SackRateAllowed','IntRateAllowed']

for stat in off_stats:
    df[f'{stat}_RankOff'] = week_grp[stat].rank(ascending=False, method='min')
for stat in def_stats:
    df[f'{stat}_RankDef'] = week_grp[stat].rank(ascending=True,  method='min')

# 8) Home/Away deltas vs. league avg at Home/Away
df['AvgPassYds_HmAway'] = df.groupby(['Year','Week','Home'])['Passing Yds'].transform('mean')
df['PassYds_vs_HmAvg']  = df['Passing Yds'] - df['AvgPassYds_HmAway']
df['AvgRushYds_HmAway'] = df.groupby(['Year','Week','Home'])['Rushing Yds'].transform('mean')
df['RushYds_vs_HmAvg']  = df['Rushing Yds'] - df['AvgRushYds_HmAway']

# 9) Compute rest days
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
df = df.sort_values(['Team','Date'])
df['DaysSince'] = df.groupby('Team')['Date'].diff().dt.days.fillna(7).astype(int)

# 10) Optional: Travel burden if you have lat/lon columns
if 'latitude' in df.columns and 'longitude' in df.columns:
    df['prev_lat'] = df.groupby('Team')['latitude'].shift(1)
    df['prev_lon'] = df.groupby('Team')['longitude'].shift(1)
    df['TravelMiles'] = df.apply(
        lambda x: geodesic((x['prev_lat'], x['prev_lon']),
                           (x['latitude'], x['longitude'])).miles
                  if pd.notnull(x['prev_lat']) else 0,
        axis=1
    )
    df.drop(columns=['prev_lat','prev_lon'], inplace=True)

# 11) Assemble your final feature DataFrame
feature_cols = (
    ['Year','Week','Team','Opp','Home'] +
    off_stats + def_stats +
    [f'{s}_RankOff' for s in off_stats] +
    [f'{s}_RankDef' for s in def_stats] +
    ['PassYds_vs_HmAvg','RushYds_vs_HmAvg','DaysSince'] +
    (['TravelMiles'] if 'TravelMiles' in df.columns else [])
)
team_weekly_features = df[feature_cols].reset_index(drop=True)

# 12) Inspect
pd.set_option('display.max_columns', None)
print(team_weekly_features.head(10))


   Year Week Team  Opp Home  Passing Yds  Rushing Yds  Total Offense Tot  \
0  2006   01  ARI  SFO    1        283.0         84.0              367.0   
1  2006   02  ARI  SEA    0        191.0         65.0              256.0   
2  2006   03  ARI  LAR    1        246.0        101.0              347.0   
3  2006   04  ARI  ATL    0        161.0         26.0              187.0   
4  2006   05  ARI  KAN    1        222.0         76.0              298.0   
5  2006   06  ARI  CHI    1        220.0         66.0              286.0   
6  2006   07  ARI  LVR    0        174.0         50.0              224.0   
7  2006   08  ARI  GNB    0        132.0         86.0              218.0   
8  2006   10  ARI  DAL    1        209.0         86.0              295.0   
9  2006   11  ARI  DET    1        228.0        110.0              338.0   

   Team Pts  PassYdsAllowed  RushYdsAllowed  PtsAllowed  SackRateAllowed  \
0        34           286.0           107.0          27         0.025000   
1        10

In [123]:
# Export the fully‐cleaned DataFrame to CSV
output_path = "/Users/sethfried/Fantasy Football/Cleaned Data/Cleaned_team_Logs/all_teams_with_opps_cleaned.csv"
df.to_csv(output_path, index=False)
print(f"Saved cleaned DataFrame to {output_path}")


Saved cleaned DataFrame to /Users/sethfried/Fantasy Football/Cleaned Data/Cleaned_team_Logs/all_teams_with_opps_cleaned.csv
